In [21]:
# Dropping duplicates 

day = 1

df_contacts = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/18-graphs-minutes-within-100m/MO_1510{day}/")

df_no_duplicates = df_contacts.drop_duplicates(subset=["graph_id","id_avl_1","id_avl_2"])

df_no_duplicates.write.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/19-graphs-minutes-within-100m-graph-no-duplicates/MO_1510{day}/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# adding column information to group the data statistics

day = 1

traces = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/19-graphs-minutes-within-100m-graph-no-duplicates/MO_1510{day}/")

df = traces.groupby("id_avl_1","graph_id").agg(F.countDistinct("id_avl_2").alias("counter_buses_connections"))

df = df.withColumn('splitted', F.split(df['graph_id'], '-'))\
    .withColumn('hour', F.col('splitted')[0])\
    .withColumn('minute', F.col('splitted')[1])\
    .withColumn('hour-minute', F.concat(F.col('hour'),F.lit(":"),F.col("minute")))\
    .withColumn('region', F.col('splitted')[2])\
    .drop("splitted")

df = df.repartition("graph_id")

df.write.parquet(f"s3://mobility-traces-sp/metrics-calculation/connectivity-metrics/vehicle-degree-per-graph/MO_1510{day}/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Degree vehicle per average minute
import pyspark.sql.functions as F
day = 1

df = spark.read.parquet(f"s3://mobility-traces-sp/metrics-calculation/connectivity-metrics/vehicle-degree-per-graph/MO_1510{day}/")
df = df.groupby("hour-minute")\
    .agg(F.avg("counter_buses_connections").alias("avg_buses_count"))\
    .withColumn('time', F.date_format('hour-minute','HH:mm'))\
    .drop("hour-minute")

df.write.parquet(f"s3://mobility-traces-sp/metrics-calculation/connectivity-metrics/vehicle-degree-per-minute/MO_1510{day}/")

In [37]:
# Degree vehicle per day
import pyspark.sql.functions as F

day = 1
count_connections = spark.read.parquet(f"s3://mobility-traces-sp/metrics-calculation/connectivity-metrics/vehicle-degree-per-graph/MO_1510{day}/")

count_connections.agg(
    F.avg("counter_buses_connections").alias("avg"),
    F.stddev("counter_buses_connections").alias("stddev"),
    F.max("counter_buses_connections").alias("max"),
    F.min("counter_buses_connections").alias("min"),
).show()
 
quantiles = count_connections.approxQuantile("counter_buses_connections", [0.25,0.5,0.75], 0.0001)
print(quantiles)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+---+---+
|              avg|            stddev|max|min|
+-----------------+------------------+---+---+
|6.246168604107917|7.7183547470265506| 94|  1|
+-----------------+------------------+---+---+

[2.0, 3.0, 7.0]

In [22]:
# When the top vertix degree happed? and Where?
import pyspark.sql.functions as F
day = 1

count_connections = spark.read.parquet(f"s3://mobility-traces-sp/metrics-calculation/connectivity-metrics/vehicle-degree-per-graph/MO_1510{day}/")

df = count_connections.filter((F.col("counter_buses_connections") > 7) & (F.col("counter_buses_connections") <= 21))
df.groupby("region").agg(F.count("id_avl_1").alias("count_register")).sort(F.col("count_register").desc()).show(96)
df.groupby("hour").agg(F.count("id_avl_1").alias("count_register")).sort(F.col("count_register").desc()).show(96)
print("vertices",count_connections.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------+
|            region|count_register|
+------------------+--------------+
|       SANTO AMARO|        131948|
|   JARDIM SAO LUIS|         87644|
|           TATUAPE|         68982|
|          ITAQUERA|         67701|
|     CAPAO REDONDO|         59642|
|            GRAJAU|         56224|
|       PARELHEIROS|         40148|
|              LAPA|         39494|
|         PINHEIROS|         39172|
|         REPUBLICA|         38813|
|         JABAQUARA|         37459|
|     JARDIM ANGELA|         36868|
|             PENHA|         35551|
|      CIDADE DUTRA|         34933|
|          PEDREIRA|         30724|
|      CACHOEIRINHA|         27567|
|           SOCORRO|         27540|
|       CAMPO LIMPO|         26798|
|        SAO MATEUS|         26576|
|           SANTANA|         24253|
|       BARRA FUNDA|         23563|
|    ITAIM PAULISTA|         23500|
|       ARTUR ALVIM|         22580|
| CIDADE TIRADENTES|         20813|
|                SE|        

In [ ]:
import pyspark.sql.functions as F

# Creating a connection_id for each connection with the buses.
# Connection_id = id_avl_1+id_avl_2 if id_avl_1 < id_avl_2
# Connection_id = id_avl_2+id_avl_1 if id_avl_2 < id_avl_1

day = 1

df = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/18-graphs-minutes-within-100m/MO_1510{day}/")

df = df.withColumn("connection_id",
    F.when(F.col("id_avl_1") < F.col("id_avl_2"), F.concat(F.col("id_avl_1"),F.lit("-"),F.col("id_avl_2")))
    .otherwise(F.concat(F.col("id_avl_2"),F.lit("-"),F.col("id_avl_1"))))

df = df.repartition("graph_id")

df.write.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/19-graphs-minutes-within-100m-with-cnnection-id/MO_1510{day}/")

In [49]:
# Number of repeated connections per day/duration of the connections

import pyspark.sql.functions as F

day = 1

df = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/19-graphs-minutes-within-100m-with-cnnection-id/MO_1510{day}/")

df = df.drop_duplicates(subset=["connection_id","graph_id"])

df_counts = df.groupby("connection_id").agg(F.count("connection_id").alias("count_per_day"))

df_counts.agg(
    F.avg("count_per_day").alias("avg"),
    F.stddev("count_per_day").alias("stddev"),
    F.max("count_per_day").alias("max"),
    F.min("count_per_day").alias("min"),
).show()
 
quantiles = df_counts.approxQuantile("count_per_day", [0.25,0.5,0.75], 0.0001)
print(quantiles)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----------------+---+---+
|              avg|           stddev|max|min|
+-----------------+-----------------+---+---+
|5.098736130373099|10.74065307404639|858|  1|
+-----------------+-----------------+---+---+

[1.0, 2.0, 5.0]

In [27]:
# Saving count of repeated connections

import pyspark.sql.functions as F

day = 12

df = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/19-graphs-minutes-within-100m-with-cnnection-id/MO_1510{day}/")

df = df.drop_duplicates(subset=["connection_id","graph_id"])

df_counts = df.groupby("connection_id").agg(F.count("connection_id").alias("count_per_day"))

df_counts.write.parquet(f"s3://mobility-traces-sp/metrics-calculation/connectivity-metrics/repeated-connection-per-day/MO_1510{day}")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Total number of distinct connections per day
import pyspark.sql.functions as F

day = 1
connections = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/19-graphs-minutes-within-100m-with-cnnection-id/MO_1510{day}/")

connections.agg(F. countDistinct("connection_id").alias("number_distinct_conn")).show()
connections.groupby("graph_id","connection_id").agg(F.countDistinct("connection_id").alias("count_novo"))\
    .agg(F.sum("count_novo").alias("total_per_day")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|number_distinct_conn|
+--------------------+
|             5021404|
+--------------------+

+-------------+
|total_per_day|
+-------------+
|     25602814|
+-------------+

In [20]:
# Total number of connections per day
import pyspark.sql.functions as F

day = 4
connections = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/19-graphs-minutes-within-100m-with-cnnection-id/MO_1510{day}/")

connections.agg(F. countDistinct("connection_id").alias("number_distinct_conn")).show()
connections.groupby("graph_id","connection_id").agg(F.countDistinct("connection_id").alias("count_novo"))\
    .agg(F.sum("count_novo").alias("total_per_day")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|number_distinct_conn|
+--------------------+
|             1132836|
+--------------------+

+-------------+
|total_per_day|
+-------------+
|      8013742|
+-------------+

In [15]:
# Total number of connections per region
import pyspark.sql.functions as F

day = 1

connections = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/19-graphs-minutes-within-100m-with-cnnection-id/MO_1510{day}/")

df = connections.groupby("hour_avl","minute_avl","region").agg(F.countDistinct("connection_id").alias("count_novo"))
# df.groupby("hour_avl").agg(F.sum("count_novo").alias("total_per_day")).show()
df.groupby("region").agg(F.sum("count_novo").alias("total_per_day"))\
    .write.parquet(f"s3://mobility-traces-sp/metrics-calculation/connectivity-metrics/region-conn/MO_1510{day}-2/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Total number of connections per hour
import pyspark.sql.functions as F

day = 4

connections = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/19-graphs-minutes-within-100m-with-cnnection-id/MO_1510{day}/")

df = connections.groupby("hour_avl","minute_avl","region").agg(F.countDistinct("connection_id").alias("count_novo"))
# df.groupby("hour_avl").agg(F.sum("count_novo").alias("total_per_day")).show()
df.groupby("hour_avl").agg(F.sum("count_novo").alias("total_per_day")).show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------+
|hour_avl|total_per_day|
+--------+-------------+
|      18|       479570|
|      21|       539537|
|      17|       470792|
|      19|       519058|
|      22|       585297|
|       6|       434444|
|       9|       432817|
|      10|       447487|
|      12|       442759|
|      11|       452778|
|      13|       445037|
|      14|       460295|
|      15|       476463|
|       7|       437406|
|       8|       426251|
|      20|       485790|
|      16|       477961|
+--------+-------------+

In [33]:
# filtering data to plot on map
import pyspark.sql.functions as F
day = 1
# connections = ["13975-14010","9134-9309","35812-35908","37220-38073","39791-40346","11665-47162","11336-11670","39297-40590"]
# connections_1 = ["9981-11048","38760-38795","11984-12402","39884-40427","9673-9745","42020-56337","40749-57946","42020-56337","11007-44874",
#                  "12006-12563",
#                 "42562-56472",
#                 "36796-42263",
#                 "13367-40693",
#                 "42857-52386",
#                 "38682-43707",
#                 "38682-43707",
#                 "35758-51310",
#                 "35205-58436",
#                 "42577-57502",
#                 "11386-38742",
#                 "8563-56600",
#                  "8565-47053",
#                 "38301-41485", 
#                 "42046-56550",
#                 "14408-37912",]
c3 = ["38682-43707",
"35758-51310",
"35205-58436",
"42577-57502",
"11386-38742",
"8563-56600",
"8565-47053",
"38301-41485",
"42046-56550",
"14408-37912"]

traces = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/19-graphs-minutes-within-100m-with-cnnection-id/MO_1510{day}/")
df_novo = traces.filter(F.col("connection_id").isin(c3)).drop_duplicates(subset=["graph_id","connection_id"])
df_novo.write.parquet(f"s3://mobility-traces-sp/metrics-calculation/connectivity-metrics/testing-plotting-map/MO_1510{day}-8/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
day = 1
df = spark.read.parquet(f"s3://mobility-traces-sp/metrics-calculation/connectivity-metrics/repeated-connection-per-day/MO_1510{day}/")

df.agg(
    F.avg("count_per_day").alias("avg"),
    F.stddev("count_per_day").alias("stddev"),
    F.max("count_per_day").alias("max"),
    F.min("count_per_day").alias("min"),
).show()
 
quantiles = df.approxQuantile("count_per_day", [0.25,0.5,0.75], 0.0001)
print(quantiles)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+---+---+
|              avg|            stddev|max|min|
+-----------------+------------------+---+---+
|5.098736130373099|10.740653074046444|858|  1|
+-----------------+------------------+---+---+

[1.0, 2.0, 5.0]

In [29]:
day = 20
df = spark.read.parquet(f"s3://mobility-traces-sp/metrics-calculation/connectivity-metrics/repeated-connection-per-day/MO_1510{day}/")

df.agg(
    F.avg("count_per_day").alias("avg"),
    F.stddev("count_per_day").alias("stddev"),
    F.max("count_per_day").alias("max"),
    F.min("count_per_day").alias("min"),
).show()
 
quantiles = df.approxQuantile("count_per_day", [0.25,0.5,0.75], 0.0001)
print(quantiles)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+---+---+
|              avg|            stddev|max|min|
+-----------------+------------------+---+---+
|5.239781710781897|11.346529878417398|850|  1|
+-----------------+------------------+---+---+

[1.0, 2.0, 5.0]

In [30]:
day = 4
df = spark.read.parquet(f"s3://mobility-traces-sp/metrics-calculation/connectivity-metrics/repeated-connection-per-day/MO_1510{day}/")

df.agg(
    F.avg("count_per_day").alias("avg"),
    F.stddev("count_per_day").alias("stddev"),
    F.max("count_per_day").alias("max"),
    F.min("count_per_day").alias("min"),
).show()
 
quantiles = df.approxQuantile("count_per_day", [0.25,0.5,0.75], 0.0001)
print(quantiles)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----------------+----+---+
|              avg|           stddev| max|min|
+-----------------+-----------------+----+---+
|7.074053084471186|26.74176288941452|1001|  1|
+-----------------+-----------------+----+---+

[1.0, 2.0, 5.0]

In [31]:
day = 12
df = spark.read.parquet(f"s3://mobility-traces-sp/metrics-calculation/connectivity-metrics/repeated-connection-per-day/MO_1510{day}/")

df.agg(
    F.avg("count_per_day").alias("avg"),
    F.stddev("count_per_day").alias("stddev"),
    F.max("count_per_day").alias("max"),
    F.min("count_per_day").alias("min"),
).show()
 
quantiles = df.approxQuantile("count_per_day", [0.25,0.5,0.75], 0.0001)
print(quantiles)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----------------+---+---+
|             avg|           stddev|max|min|
+----------------+-----------------+---+---+
|7.45246349483829|29.10027859255961|987|  1|
+----------------+-----------------+---+---+

[1.0, 2.0, 6.0]

In [32]:
day = 17
df = spark.read.parquet(f"s3://mobility-traces-sp/metrics-calculation/connectivity-metrics/repeated-connection-per-day/MO_1510{day}/")

df.agg(
    F.avg("count_per_day").alias("avg"),
    F.stddev("count_per_day").alias("stddev"),
    F.max("count_per_day").alias("max"),
    F.min("count_per_day").alias("min"),
).show()
 
quantiles = df.approxQuantile("count_per_day", [0.25,0.5,0.75], 0.0001)
print(quantiles)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----------------+---+---+
|              avg|           stddev|max|min|
+-----------------+-----------------+---+---+
|6.429004381135884|20.39699195599561|994|  1|
+-----------------+-----------------+---+---+

[1.0, 2.0, 5.0]

In [5]:
# Verifying timestamps of connections
import pyspark.sql.functions as F

connection = "14408-37912"
traces = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/19-graphs-minutes-within-100m-with-cnnection-id/MO_1510{1}/")
traces.drop_duplicates(subset=["graph_id","connection_id"]).filter(f"connection_id == '11984-12402'").select("hour_avl","minute_avl","region").sort(F.col("hour_avl"),F.col("minute_avl")).show(1000,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+-------+
|hour_avl|minute_avl|region |
+--------+----------+-------+
|19      |29        |JAGUARA|
|19      |30        |JAGUARA|
|19      |31        |JAGUARA|
|19      |32        |JAGUARA|
|19      |33        |JAGUARA|
|19      |34        |JAGUARA|
|19      |36        |JAGUARA|
|19      |37        |JAGUARA|
|19      |39        |JAGUARA|
|19      |40        |JAGUARA|
|19      |41        |JAGUARA|
|19      |42        |JAGUARA|
|19      |43        |JAGUARA|
|19      |44        |JAGUARA|
|19      |45        |JAGUARA|
|19      |47        |JAGUARA|
|19      |48        |JAGUARA|
|19      |49        |JAGUARA|
|19      |50        |JAGUARA|
|19      |51        |JAGUARA|
|19      |52        |JAGUARA|
|19      |53        |JAGUARA|
|19      |54        |JAGUARA|
|19      |55        |JAGUARA|
|19      |56        |JAGUARA|
|19      |57        |JAGUARA|
|19      |58        |JAGUARA|
|19      |59        |JAGUARA|
|20      |0         |JAGUARA|
|20      |1         |JAGUARA|
|20      |

In [ ]:
diferentes-bairros-14408-37912.png --> momentos diferentes do dia e em regioes distintas (12:15 --> Socorro, 13:21 --> Cidade Dutra, 10:31 --> Socorro)

    
curto-espaco-de-tempo - 9:17,9:18,9:20  - na regiao de vila mariana  
encontro-8565-47053 - Se encontram as 17:14, 13:50, 9:27 em sao lucas
lugares-diferente - 16:26/16:28,16:30, 32, tatuape, 9:49 carrao
terminal rua 42020-56337 - se encontram as 8,7,16,20,14 itaquera

11984-12402 -> 20,21,19 jaguara   



In [44]:
traces = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/19-graphs-minutes-within-100m-with-cnnection-id/MO_1510{1}/")
df = traces.groupby("id_avl_1").agg(F.countDistinct("id_avl_2").alias("diff_vehicles"))
df
df.agg(
    F.avg("diff_vehicles").alias("avg"),
    F.stddev("diff_vehicles").alias("stddev"),
    F.max("diff_vehicles").alias("max"),
    F.min("diff_vehicles").alias("min"),
).show()
 
quantiles = df.approxQuantile("diff_vehicles", [0.25,0.5,0.75], 0.0001)
print(quantiles)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------------------+----+---+
|              avg|            stddev| max|min|
+-----------------+------------------+----+---+
|717.2409655763462|338.69108484314785|1964|  1|
+-----------------+------------------+----+---+

[470.0, 691.0, 925.0]